LSTM

In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv('appliance_usage_data2.csv')

# Convertir la columna "Fecha" en tipo datetime
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Desglosar la fecha en características numéricas
data['Year'] = data['Fecha'].dt.year
data['Month'] = data['Fecha'].dt.month
data['Day'] = data['Fecha'].dt.day
data['Hour'] = data['Fecha'].dt.hour

X = data[['Year', 'Month', 'Day', 'Hour', 'Watts usados', 'Porcentaje de uso', 'Porcentaje de fallos']]

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

y = data["Reporte de fallo"]

model = Sequential()

# Capa LSTM con return_sequences=True para que devuelva secuencias en lugar de un solo valor
model.add(LSTM(64, input_shape=(X_scaled.shape[1], 1), return_sequences=True))

model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64, return_sequences=True))

# Capa LSTM final que no devuelve secuencias
model.add(LSTM(64))

# Capa densa de salida con función de activación sigmoide para la clasificación binaria
model.add(Dense(1, activation='sigmoid'))

# Compilación del modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(X_scaled, y, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10


KeyboardInterrupt: 

Predicciones del modelo

In [ ]:
# Cargar tus datos de prueba (asegúrate de que tengan la misma estructura que los datos de entrenamiento)
test_data = pd.read_csv('appliance_usage_data.csv')
X_test = test_data[['Hora', 'Watts usados', 'Porcentaje de fallos']]
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Cargar el modelo entrenado
model = load_model('lstm_model2.h5')

# Realizar predicciones
yhat = model.predict(X_test)

print(yhat)

# Aplicar un umbral (opcional) para obtener valores binarios (0's y 1's)
yhat_binary = (yhat >= 0.5).astype(int)

15625/15625 [==============================] - 30s 2ms/step
[[0.498172  ]
 [0.4961025 ]
 [0.49400753]
 ...
 [0.49392575]
 [0.4933683 ]
 [0.49557686]]


LSTM

In [4]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib

# Carga tus datos desde un archivo CSV
data = pd.read_csv('appliance_usage_data2.csv')

# Preprocesamiento de datos
X = data[['Hora', 'Watts usados', 'Porcentaje de fallos']].values
y = data['Reporte de fallo'].values

# Normalizar los datos de entrada
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

# Normalizar los datos de salida
scaler_y = MinMaxScaler(feature_range=(0, 1))
y = scaler_y.fit_transform(y.reshape(-1, 1))

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cambiar la forma de los datos para LSTM (muestras, pasos de tiempo, características)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Crear el modelo LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False)

# Guardando el modelo
model.save('lstm_model.h5')

# Guardando los escaladores
joblib.dump(scaler, 'scaler_lstm.pkl')
joblib.dump(scaler_y, 'scaler_y_lstm.pkl')

# Evaluar el modelo
y_pred = model.predict(X_test)

# Invertir la normalización en las predicciones
y_pred_orig_scale = scaler_y.inverse_transform(y_pred)

print(y_pred_orig_scale)

# Mostrar la precisión del modelo
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_test, y_pred))

print(f'RMSE: {rmse}')


Epoch 1/10
5556/5556 - 23s - loss: 0.5001 - val_loss: 0.4998 - 23s/epoch - 4ms/step
Epoch 2/10
5556/5556 - 20s - loss: 0.5001 - val_loss: 0.4998 - 20s/epoch - 4ms/step
Epoch 3/10
5556/5556 - 18s - loss: 0.5000 - val_loss: 0.4997 - 18s/epoch - 3ms/step
Epoch 4/10
5556/5556 - 18s - loss: 0.5000 - val_loss: 0.4997 - 18s/epoch - 3ms/step
Epoch 5/10
5556/5556 - 17s - loss: 0.4999 - val_loss: 0.4996 - 17s/epoch - 3ms/step
Epoch 6/10
5556/5556 - 17s - loss: 0.4999 - val_loss: 0.4996 - 17s/epoch - 3ms/step
Epoch 7/10
5556/5556 - 17s - loss: 0.4999 - val_loss: 0.4996 - 17s/epoch - 3ms/step
Epoch 8/10
5556/5556 - 16s - loss: 0.4999 - val_loss: 0.4996 - 16s/epoch - 3ms/step
Epoch 9/10
5556/5556 - 16s - loss: 0.4999 - val_loss: 0.4996 - 16s/epoch - 3ms/step
Epoch 10/10
5556/5556 - 16s - loss: 0.4999 - val_loss: 0.4996 - 16s/epoch - 3ms/step


c:\Users\sebas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3125/3125 [==============================] - 6s 2ms/step
[[0.04937335]
 [0.04891926]
 [0.04676594]
 ...
 [0.04745354]
 [0.04797097]
 [0.04860092]]
RMSE: 0.6734465895755429


Haciendo predicciones

In [26]:
import pandas as pd
import numpy as np
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import joblib

# Cargando los escaladores.
scaler = joblib.load('scaler_lstm.pkl')
scaler_y = joblib.load('scaler_y_lstm.pkl')

# Carga tus datos desde un archivo CSV
data = pd.read_csv('appliance_usage_data1.csv')

# Selecciona las columnas que quieres usar como datos de entrada
input_data = data[['Hora', 'Watts usados', 'Porcentaje de fallos']].values

input_data = scaler.transform(input_data)

# Cambiar la forma de los datos para LSTM (muestras, pasos de tiempo, características)
input_data = input_data.reshape((input_data.shape[0], 1, input_data.shape[1]))

# Cargar el modelo
model = load_model('lstm_model.h5')

# Hacer una predicción
predictions = model.predict(input_data)

# Invirtiendo la normalización en las predicciones
predictions_orig_scale = scaler_y.inverse_transform(predictions)

# Convertir las predicciones a 0's y 1's
y_pred_binary = np.where(y_pred > 0.05, 1, 0)

print(y_pred_binary)

# Guardar las predicciones en un archivo txt
with open('predictions.txt', 'w') as f:
    for prediction in y_pred_binary:
        f.write(f'{prediction}\n')

15625/15625 [==============================] - 30s 2ms/step
[[1]
 [1]
 [0]
 ...
 [0]
 [1]
 [1]]


RNN

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
import numpy as np

data = pd.read_csv('appliance_usage_data2.csv')

# Preprocesamiento de los datos
data['Fecha'] = pd.to_datetime(data['Fecha'])
data['Year'] = data['Fecha'].dt.year
data['Month'] = data['Fecha'].dt.month
data['Day'] = data['Fecha'].dt.day
data['Hour'] = data['Fecha'].dt.hour

X = data[['Year', 'Month', 'Day', 'Hour', 'Watts usados', 'Porcentaje de uso', 'Porcentaje de fallos']]
y = data["Reporte de fallo"]

# Normalización de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cambio de forma de los datos para el modelo RNN
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

print(X_train.shape, X_test.shape)

# Definición del modelo RNN
model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(1, X_train.shape[2]), activation="relu"))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=5, batch_size=16, verbose=2)

# Evaluación del modelo
loss = model.evaluate(X_test, y_test)
print('Loss:', loss)

# Guardar el modelo
model.save('rnn_model.h5')

(400000, 1, 7) (100000, 1, 7)
Epoch 1/5
25000/25000 - 123s - loss: 0.2515 - 123s/epoch - 5ms/step
Epoch 2/5
25000/25000 - 109s - loss: 0.2508 - 109s/epoch - 4ms/step
Epoch 3/5
25000/25000 - 115s - loss: 0.2506 - 115s/epoch - 5ms/step
Epoch 4/5
25000/25000 - 93s - loss: 0.2505 - 93s/epoch - 4ms/step
Epoch 5/5
25000/25000 - 51s - loss: 0.2504 - 51s/epoch - 2ms/step
3125/3125 [==============================] - 6s 2ms/step - loss: 0.2502
Loss: 0.2502404749393463


c:\Users\sebas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
# Haciendo una predicción con los X_test y Y_test.
y_pred = model.predict(X_test)

# Mostrar la precisión del modelo
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(y_test, y_pred))

print("RMSE: ", rmse)

3125/3125 [==============================] - 5s 2ms/step
RMSE:  0.5001593783747433


In [6]:
from keras.models import load_model
from sklearn.metrics import accuracy_score
import pandas as pd


model = load_model('rnn_model.h5')

# Nueva entrada
new_input = pd.read_csv('appliance_usage_data1.csv')

new_input['Fecha'] = pd.to_datetime(new_input['Fecha'])
new_input['Year'] = new_input['Fecha'].dt.year
new_input['Month'] = new_input['Fecha'].dt.month
new_input['Day'] = new_input['Fecha'].dt.day
new_input['Hour'] = new_input['Fecha'].dt.hour

new_input = new_input[['Year', 'Month', 'Day', 'Hour', 'Watts usados', 'Porcentaje de uso', 'Porcentaje de fallos']]

# Normalizar la nueva entrada
new_input_scaled = scaler.transform(new_input)

# Cambiar la forma de la nueva entrada para que sea adecuada para el modelo RNN
new_input_scaled = np.reshape(new_input_scaled, (new_input_scaled.shape[0], 1, new_input_scaled.shape[1]))

# Hacer la predicción
prediction = model.predict(new_input_scaled)

print('Predicción:', prediction)

predictions = [1 if p > 0.5 else 0 for p in prediction]

#  # Contando cuántas veces se predijo un fallo
# count = 0
# for prediction in predictions:
#     if prediction == 1:
#         count += 1

#print('Número de veces que se predijo un fallo:', count)

# Midiendo el accuracy
data = pd.read_csv('appliance_usage_data1.csv')

y_true = data['Reporte de fallo']

# Calcular la precisión del modelo
accuracy = accuracy_score(y_true, predictions)

print('Precisión:', accuracy)

# # Aplicar la función de umbral
# prediction = [1 if p > 0.5 else 0 for p in prediction]

# print('Predicción:', prediction)

# Armando la matriz de confusión
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, predictions)

print(cm)

# Calcular la media de las predicciones
mean_prediction = np.mean(prediction)

# Calcular la desviación estándar de las predicciones
std_deviation_prediction = np.std(prediction)

print('Media de las predicciones:', mean_prediction)
print('Desviación estándar de las predicciones:', std_deviation_prediction)

15625/15625 [==============================] - 27s 2ms/step
Predicción: [[0.48505282]
 [0.49314326]
 [0.48530567]
 ...
 [0.49048126]
 [0.5018786 ]
 [0.4764734 ]]
Precisión: 0.5
[[244287   5566]
 [244434   5713]]
Media de las predicciones: 0.48536012
Desviación estándar de las predicciones: 0.0067249285
